In [1]:
import pandas as pd
import numpy as np
import datetime
import timing
from sklearn.cluster import AgglomerativeClustering

In [2]:
path = desktop = '/Users/panbingqing/Documents/工作/城市大脑/长沙调控/data/'

# 获取数据

### 获取异常处理填充后的数据

In [3]:
fill_data = pd.read_csv('/Users/panbingqing/Documents/工作/城市大脑/长沙调控/data_inline/fill_data.csv')
fill_data.passtime = pd.to_datetime(fill_data.passtime, format='%Y/%m/%d %H:%M:%S')
fill_data.head()

,camera_id,car_count,date,direction,inter_name,lane,passtime,road_name,time,weekday
0,1,3.0,2019-05-17,LB,星沙-滨湖,1,2019-05-17 00:00:00,星沙-滨湖北,00:00:00,Friday
1,1,6.0,2019-05-24,LB,星沙-滨湖,1,2019-05-24 00:00:00,星沙-滨湖北,00:00:00,Friday
2,1,1.0,2019-05-31,LB,星沙-滨湖,1,2019-05-31 00:00:00,星沙-滨湖北,00:00:00,Friday
3,1,2.0,2019-06-14,LB,星沙-滨湖,1,2019-06-14 00:00:00,星沙-滨湖北,00:00:00,Friday
4,1,1.0,2019-05-17,LB,星沙-滨湖,1,2019-05-17 00:03:00,星沙-滨湖北,00:03:00,Friday


# 早晚高峰时段判定

In [4]:
# def get_date_mean_by_weekday(data):
#     data = data.copy()
#     data.index = data.passtime
#     data = data.groupby(['inter_name', 'road_name', 'weekday']).resample('15T').car_count.sum().reset_index()
#     data['date'] = data.passtime.dt.date
#     data['time'] = data.passtime.dt.time
#     data = data.groupby(['inter_name', 'road_name', 'time', 'weekday']).car_count.mean()
#     data = data.groupby(['inter_name', 'time', 'weekday']).sum().reset_index()
#     return data
    
# weekday_data_15T = get_date_mean_by_weekday(fill_data)
# weekday_data_15T.head()

In [5]:
# def get_peak_range(data, peak_time, is_morning=True):
#     # 获取高峰时段范围
#     data = data[['time', 'car_count']].sort_values(
#         'time').reset_index(drop=True).copy()
    
#     time_range = ([datetime.time(7, 0), datetime.time(10, 0)]
#                   if is_morning
#                   else [datetime.time(16, 0), datetime.time(22, 0)])
    
#     range_data = data[(data.time>=time_range[0]) &
#                       (data.time<time_range[1])]
    
#     peak_data = data[(data.time>=peak_time[0]) &
#                      (data.time<peak_time[1])]
# #     down_limit = peak_data.car_count.max() - (
# #         peak_data.car_count.max() - peak_data.car_count.min()) * 1.2
#     down_limit = data[data.time == datetime.time(7, 15)].car_count.values[0]
    
#     peak_range = range_data[range_data.car_count >= down_limit].index
#     return data.loc[peak_range[0], 'time'], data.loc[peak_range[-1]+1, 'time']

# def get_peak(data):
#     # 获取多天高峰时段
#     peak_times = []
    
#     morning_peak_time = [datetime.time(7, 30), datetime.time(9, 0)]
#     evening_peak_time = [datetime.time(17, 0), datetime.time(19, 0)]
#     for name, group in data.groupby(['inter_name', 'weekday']):
#         peak_time = {}
#         peak_time['inter_name'] = name[0]
#         peak_time['weekday'] = name[1]
#         peak_time['morning'] = get_peak_range(group, morning_peak_time)
#         peak_time['evening'] = get_peak_range(group, evening_peak_time, is_morning=False)

#         peak_times.append(peak_time)
#     return peak_times

In [6]:
# peak_times = get_peak(weekday_data_15T)
# peak_data = weekday_data_15T.copy()
# peak_data['is_peak'] = None
# for peak_time in peak_times:
#     for _range in ['morning', 'evening']:
#         peak_data.loc[(
#             (peak_data.inter_name==peak_time['inter_name']) &
#             (peak_data.weekday==peak_time['weekday']) &
#             (peak_data.time>=peak_time[_range][0]) &
#             (peak_data.time<peak_time[_range][1])
#         ), 'is_peak'] = _range
#     peak_data.loc[(
#         (peak_data.inter_name==peak_time['inter_name']) &
#         (peak_data.weekday==peak_time['weekday']) &
#         (peak_data.time>=peak_time['morning'][1]) &
#         (peak_data.time<peak_time['evening'][0])
#     ), 'is_peak'] = 'flat'

# 数据筛选

In [7]:
# 天气数据
def get_sunny_date(path):
    weather = pd.read_csv(path+'长沙天气.csv')
    weather.date = pd.to_datetime(weather.date, format='%Y/%m/%d').dt.date
    sunny_date = weather.loc[(~weather.weather.str.contains('雨')), 'date']
    return sunny_date

In [8]:
# 重采样
def resample_data(data, sep='6T'):
    # 6分钟重采样
    data = data.copy()
    data['date'] = data.passtime.dt.date.astype(str)
    data.index = data.pop('passtime')
    data = data.groupby(
        ['inter_name', 'road_name', 'direction', 'lane', 'weekday', 'date']
    ).resample(sep).car_count.sum().reset_index()
    data['time'] = data.passtime.dt.time
    return data

def sel_by_time(data, stime, etime):
    # 时间筛选
    data = data[(data.time >= stime) &
                (data.time < etime)]
    return data
    

In [9]:
sunny_date = get_sunny_date(path)

# 星沙-望仙/晴天筛选
sel_data = fill_data[
    (fill_data.inter_name == '星沙-望仙') &
    (fill_data.passtime.dt.date.isin(sunny_date))
].reset_index(drop=True).copy()

# 路口全量分析

In [10]:
veh_count = resample_data(sel_data, sep='15T')
road_count = veh_count.groupby(['inter_name', 'road_name', 'weekday', 'date', 'time']).car_count.sum().reset_index()
weekday_mean_count = road_count.groupby(['inter_name', 'road_name', 'weekday', 'time']).car_count.mean().reset_index()
weekday_mean_count = weekday_mean_count.groupby(['inter_name', 'weekday', 'time']).car_count.sum().reset_index()
weekday_mean_count['date'] = 'date_mean'

road_count = road_count.groupby(['inter_name', 'weekday', 'date', 'time']).car_count.sum().reset_index()
road_count = road_count.append(weekday_mean_count, sort=True)
road_count.head()

,car_count,date,inter_name,time,weekday
0,237.0,2019-05-17,星沙-望仙,00:00:00,Friday
1,237.0,2019-05-17,星沙-望仙,00:15:00,Friday
2,234.0,2019-05-17,星沙-望仙,00:30:00,Friday
3,209.0,2019-05-17,星沙-望仙,00:45:00,Friday
4,225.0,2019-05-17,星沙-望仙,01:00:00,Friday


In [11]:
road_count.to_csv(desktop+'road_count.csv', index=False)

# 相关性

In [12]:
relations_by_date = []
for name, group in road_count.groupby(['inter_name', 'weekday']):
    for _date in group.date.unique():
        if _date == 'date_mean':
            continue
        relation = {'inter_name': name[0], 'weekday': name[1]}
        pivot_table = pd.pivot_table(
            group[group.date.isin([_date, 'date_mean'])], index='time', columns='date', values='car_count')
        relation['date'] = _date
        relation['corr'] = pivot_table.corr().values[0][1]
        relation['var_coef'] = pivot_table.std(axis=1).mean() / pivot_table['date_mean'].mean()
        relations_by_date.append(relation)
        
relations_by_date = pd.DataFrame(relations_by_date)

In [13]:
relations_by_date.to_csv(path+'relation_by_date.csv', index=False)

In [14]:
peak_of_weekday = {
    'Monday': [[datetime.time(7, 15), datetime.time(9, 30)], [datetime.time(16, 15), datetime.time(21, 45)]],
    'Tuesday': [[datetime.time(7, 15), datetime.time(10, 0)], [datetime.time(16, 30), datetime.time(21, 45)]],
    'Wednesday': [[datetime.time(7, 15), datetime.time(10, 0)], [datetime.time(16, 0), datetime.time(21, 15)]],
    'Thursday': [[datetime.time(7, 30), datetime.time(9, 0)], [datetime.time(16, 0), datetime.time(21, 30)]],
    'Friday': [[datetime.time(7, 15), datetime.time(11, 0)], [datetime.time(13, 15), datetime.time(22, 45)]],
    'Saturday': [[datetime.time(7, 30), datetime.time(22, 15)]],
    'Sunday': [[datetime.time(7, 30), datetime.time(22, 15)]]
}

In [15]:
relations_by_weekday = pd.DataFrame()
for name, group in weekday_mean_count.groupby(['inter_name']):
    relations = pd.DataFrame()
    weekdays = group.weekday.unique()
    for weekday_x in weekdays:
        for peak_time in peak_of_weekday[weekday_x]:
            stime, etime = peak_time
            data = group[(group.time>=stime) &
                         (group.time<etime)]
            pivot_table = pd.pivot_table(
                data, index='time', columns='weekday', values='car_count')
            relation = pd.DataFrame(pivot_table.corr()[weekday_x])
            relation.columns = ['corr']
            relation['coef_var'] = 0
            
            for weekday_y in weekdays:
                relation.loc[weekday_y, 'coef_var'] = (
                    pivot_table[[weekday_x, weekday_y]].std(axis=1).mean() /
                    pivot_table[weekday_x].mean())
            
            relation['weekday_x'] = weekday_x
            relation['peak_range'] = str(stime) + "-" + str(etime)
            relations = relations.append(relation)
    relations['inter_name'] = name
    relations_by_weekday = relations_by_weekday.append(relations)
relations_by_weekday = relations_by_weekday.reset_index().rename(columns={'weekday': 'weekday_y'})

In [16]:
relations_by_weekday.to_csv(path+'relations_by_weekday.csv', index=False)

# 配时计算

### 获取weekday_label，及根据weekday_label求算均值

In [17]:
veh_count = resample_data(sel_data, '6T')
veh_count['weekday_label'] = veh_count.weekday

new_weekday_label = veh_count[
    veh_count.weekday.isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday'])
].copy()
new_weekday_label['weekday_label'] = 'Monday_Thursday'

veh_count = veh_count.append(new_weekday_label, sort=True)

weekday_label_mean = veh_count.groupby(
    ['inter_name', 'road_name', 'direction', 'lane', 'time', 'weekday_label']
    ).car_count.mean().reset_index(name='car_count')
weekday_label_mean['date'] = 'date_mean'

weekday_mean = weekday_label_mean[
    weekday_label_mean.weekday_label!='Monday_Thursday'
].groupby(
    ['inter_name', 'road_name', 'direction', 'lane', 'time', 'date']
    ).car_count.mean().reset_index(name='car_count')
weekday_mean['weekday_label'] = 'weekday_mean'

new_veh_count = veh_count[
    ['inter_name', 'road_name', 'direction', 'lane', 'time', 'weekday_label', 'date', 'car_count']
    ].append(weekday_label_mean, ignore_index=True, sort=True
            ).append(weekday_mean, ignore_index=True, sort=True)


### merge相位信息

In [18]:
def get_phase_info(path):
    # 获取相位信息
    phase_info = pd.read_csv('/Users/panbingqing/Documents/工作/城市大脑/长沙调控/lane_info.csv')[['road_name', 'direction', 'phase']]
    phase_info.drop_duplicates(['road_name', 'direction'], inplace=True)
    phase_info.dropna(inplace=True)
    return phase_info

In [19]:
phase_info = get_phase_info('data/info/')
phase_data = new_veh_count.merge(phase_info, how='left', on=['road_name', 'direction'])
# 不受信号灯控制的，相位名称以R表示
phase_data.phase.fillna('R', inplace=True)

In [20]:
phase_data.to_csv(path+'phase_data.csv', index=False)

### 关键相位流量

In [21]:
# 关键相位流量为相位中单车道最大值
key_phase_data = phase_data.groupby(
    ['inter_name', 'time', 'weekday_label', 'date', 'phase']
    ).car_count.max().reset_index()

In [ ]:
def fill_key_phase(key_phase, columns):
    # key_phase nan 值填充
    groups =  key_phase.groupby(columns)
    key_phase = pd.DataFrame()
    for name, group in groups:
        group = group.pivot_table(index='time', columns='phase', values='car_count')
        group.fillna(method='pad', inplace=True)
        group = group.unstack().reset_index()
        for i, col in enumerate(columns):
            group[col] = name[i]
        group.rename(columns={0: 'car_count'}, inplace=True)
        key_phase = key_phase.append(group)
    return key_phase
key_phase_data = fill_key_phase(key_phase_data, ['inter_name', 'weekday_label', 'date'])
key_phase_data.head()

In [ ]:
key_phase_data.to_csv(path+'key_phase_data.csv', index=False)

In [ ]:
key_phase_data.info()

## 配时计算

In [ ]:
# 配时计算根据weekday_label均值进行计算
key_phase_data_weekday = key_phase_data[
    (key_phase_data.date == 'date_mean') & 
    (key_phase_data.phase != 'R')
].reset_index(drop=True).copy()
key_phase_data_weekday['sat_flow_ratio'] = 1440 / 60 * 6

timing_data = pd.DataFrame()
groups = key_phase_data_weekday.groupby(['inter_name', 'time', 'weekday_label'])
for name, group in groups:
    data = group.reset_index(drop=True)
    phase_data = timing.timing(data)
    data['phase_time'] = phase_data.phase_time
    data['cycle'] = phase_data.cycle
    timing_data = timing_data.append(data)
timing_data.reset_index(drop=True, inplace=True)

### merge peak time

In [ ]:
peak_of_weekday = {
    'Monday': {'morning_peak': [datetime.time(7, 15), datetime.time(9, 30)],
               'day_flat': [datetime.time(9, 30), datetime.time(16, 15)],
               'evening_peak': [datetime.time(16, 15), datetime.time(21, 45)]},
    'Tuesday': {'morning_peak': [datetime.time(7, 15), datetime.time(10, 0)], 
                'day_flat': [datetime.time(10, 0), datetime.time(16, 30)],
                'evening_peak': [datetime.time(16, 30), datetime.time(21, 45)]},
    'Wednesday': {'morning_peak': [datetime.time(7, 15), datetime.time(10, 0)],
                  'day_flat': [datetime.time(10, 0), datetime.time(16, 0)],
                  'evening_peak': [datetime.time(16, 0), datetime.time(21, 15)]},
    'Thursday': {'morning_peak': [datetime.time(7, 30), datetime.time(9, 0)],
                 'day_flat': [datetime.time(9, 0), datetime.time(16, 0)],
                 'evening_peak': [datetime.time(16, 0), datetime.time(21, 30)]},
    'Friday': {'morning_peak': [datetime.time(7, 15), datetime.time(11, 0)],
               'day_flat': [datetime.time(11, 0), datetime.time(13, 15)],
               'evening_peak': [datetime.time(13, 15), datetime.time(22, 45)]},
    'Saturday': {'day_flat': [datetime.time(7, 30), datetime.time(22, 15)]},
    'Sunday': {'day_flat': [datetime.time(7, 30), datetime.time(22, 15)]},
    'Monday_Thursday': {'morning_peak': [datetime.time(7, 15), datetime.time(9, 30)],
                        'day_flat': [datetime.time(9, 30), datetime.time(16, 15)],
                        'evening_peak': [datetime.time(16, 15), datetime.time(21, 45)]},
}

In [ ]:
timing_data['peak_type'] = 'evening_flat'
for weekday, peak_times in peak_of_weekday.items():
    for _type, _time in peak_times.items():
        timing_data.loc[(timing_data.weekday_label == weekday) &
                        (timing_data.time >= _time[0]) &
                        (timing_data.time < _time[1]), 'peak_type'] = _type

### 根据相位时间、周期长进行分层聚类，筛选time_label

In [ ]:
def get_cluster(data, n_clusters=6, adjust_label=True):
    '''
    adjust_label: boolean; 对离散的label取临近调整
    '''
    cluster_data = data.copy()
    cluster_data.index = cluster_data.pop('time')
    
    # 根据相位时间和时间顺序进行聚类
    x_data = cluster_data.pivot_table(index='time', columns='phase', values='phase_time')
    x_data['n'] = [x * 6 for x in range(len(x_data))]
#         cycle = label_data.groupby('time').cycle.mean()
#         data = pd.DataFrame({'cycle': cycle,
#                              'n': [x+1 for x in range(len(cycle))]})
    cluster_label = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward').fit(x_data).labels_

    adjust_label = True
    while adjust_label:
        adjust_label = 0
        for i in range(1, len(x_data)-1):
            if (cluster_label[i] != cluster_label[i-1]) and (cluster_label[i] != cluster_label[i+1]):
                cluster_label[i] = cluster_label[i-1]
                adjust_label += 1

    cluster_data['cluster_label'] = pd.Series(cluster_label, index=x_data.index)
    return cluster_data.reset_index()

In [ ]:
groups = timing_data.groupby(['inter_name', 'weekday_label', 'peak_type'])
cluster_datas = pd.DataFrame()
for name, group in groups:
    n_clusters = 1
    if name[2] in ['morning_peak', 'evening_peak']:
        n_clusters = 5
    cluster_data = get_cluster(group, n_clusters)
    cluster_datas = cluster_datas.append(cluster_data)
cluster_datas.reset_index(drop=True, inplace=True)
cluster_datas.head()

In [ ]:
groups = cluster_datas.groupby(['inter_name', 'weekday_label', 'peak_type', 'cluster_label', 'phase'])
new_cluster_datas = pd.DataFrame()
for name, group in groups:
    data = group.copy()
    data['cluster_phase_time'] = timing.label_mean(data.phase_time)
    new_cluster_datas = new_cluster_datas.append(data)
new_cluster_datas.reset_index(drop=True, inplace=True)
new_cluster_datas.head()

In [ ]:
new_cluster_datas.to_csv(desktop+'timing_data.csv', index=False)